In [1]:
from binance.client import Client
import pandas as pd
import json
import numpy as np
from sklearn.linear_model import LinearRegression

In [2]:
from security import get_keys
binance_key = get_keys('client')
client = Client(binance_key['api_key'], binance_key['api_secret'])

correct password


In [3]:
all_symbol_ticker = pd.DataFrame(client.get_symbol_ticker())
all_balances = pd.DataFrame(client.get_account()['balances'])

In [16]:
all_symbol_ticker['price'].astype(float).mean()

324358.7327868959

In [4]:
symbols = ['USDT', 'BTC', 'ETH', 'XRP']
expected_pairs = []
for sym1 in symbols:
    for sym2 in symbols:
        if sym1 == sym2:
            continue
        expected_pairs.append(sym1 + sym2)
pairs = [x for x in all_symbol_ticker['symbol'] if x in expected_pairs]
pairs

['ETHBTC', 'BTCUSDT', 'ETHUSDT', 'XRPBTC', 'XRPETH', 'XRPUSDT']

In [43]:
def show_average(min_num):
    for pair in pairs:
        if 'USDT' in pair:
            data = client.get_historical_klines(pair, client.KLINE_INTERVAL_1MINUTE, str(min_num) + " min ago UTC")
            disp = pd.DataFrame(data, columns = ['Open time',
                                                'Open',
                                                'High',
                                                'Low',
                                                'Close',
                                                'Volume',
                                                'Close time',
                                                'Quote asset volume',
                                                'Number of trades',
                                                'Taker buy base asset volume',
                                                'Taker buy quote asset volume',
                                                'Can be ignored',])
            y = np.array(disp['Close'].astype(float))
            x = np.array(disp['Close time'].astype(float)).reshape((-1, 1))
            slope = LinearRegression().fit(x, y).coef_
            print(pair[:len('USDT')-1], slope[0], 'USDT/min')

In [62]:
show_average(11)

BTC 4.628787878788315e-06 USDT/min
ETH 1.2681818181818358e-06 USDT/min
XRP 4.757575757575597e-10 USDT/min
